In [1]:
import os
import numpy as sp
import pandas as pd
import mysql.connector as mysql

### Import a CSV file into a Pandas dataframe

In [2]:
df = pd.read_csv('datasets/Customer Contracts$.csv')
df

,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-19,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-15,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-12,12-20-2015,NaN,No,Yes
5,Facebook,04-29-2011,NaN,7.58,Yes,No
6,Apple,10-15-2009,NaN,NaN,Yes,Yes
7,Airbnb,04-02-08,NaN,9.42,No,No
8,Nest,09-19-2006,NaN,3.00,Yes,Yes
9,Canon,03-07-05,09-20-2009,11.26,Yes,No


### Clean the table name and remove extra symbols, spaces, capital letters

In [3]:
file = 'Customer Contracts$'
clean_tbl_name = file.lower().replace(" ", "_").replace("-","_").\
                    replace(r"/","_").replace("\\","_").\
                    replace("$","").replace("%","")
clean_tbl_name

'customer_contracts'

### Clean column headers and remove extra symbols, spaces, capital letters

In [4]:
df.columns

Index(['customer_name', 'start_date', 'end_date', 'contract_amount_m',
       'invoice_sent', 'paid'],
      dtype='object')

In [5]:
for i in df.columns:
    new_name = i.lower().replace(" ", "_").replace("-","_").\
                    replace(r"/","_").replace("\\","_").\
                    replace("$","").replace("%","")
    df.rename(columns = {'i':'new_name'}, inplace = True)

### Write the create table SQL statement

In [6]:
replacements = {
    'object': 'VARCHAR(255)',
    'float64': 'FLOAT',
    'int64': 'INT',
    'datetime64': 'DATETIME',
    'timedelta64[ns]': 'VARCHAR(255)'
}

In [7]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip (df.columns, df.dtypes.replace(replacements)))
print(col_str)

customer_name VARCHAR(255), start_date VARCHAR(255), end_date VARCHAR(255), contract_amount_m FLOAT, invoice_sent VARCHAR(255), paid VARCHAR(255)


### Connect to the database

In [8]:
db = mysql.connect(
    host = "127.0.0.1",
    user = "root",
    passwd = "root",
    database = 'csvtodbauto'
)
print(db) 
print()
cursor = db.cursor()
print('Opened database successfully')


Opened database successfully


### Create database (only do this if you are doing this project for the first time)

In [9]:
cursor.execute("CREATE DATABASE IF NOT EXISTS csvtodbauto")

### Create table

In [10]:
cursor.execute("DROP TABLE IF EXISTS %s" % (clean_tbl_name))

In [11]:
print('Creating table....')
cursor.execute("CREATE TABLE %s (%s);" % (clean_tbl_name, col_str))
print("Table is created....")

Creating table....
Table is created....


### Populate table

In [12]:
df

,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-19,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-15,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-12,12-20-2015,NaN,No,Yes
5,Facebook,04-29-2011,NaN,7.58,Yes,No
6,Apple,10-15-2009,NaN,NaN,Yes,Yes
7,Airbnb,04-02-08,NaN,9.42,No,No
8,Nest,09-19-2006,NaN,3.00,Yes,Yes
9,Canon,03-07-05,09-20-2009,11.26,Yes,No


In [13]:
df = df.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna('N/A'))
df

,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-19,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,N/A,3.90,No,No
2,Adidas,12-07-15,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-12,12-20-2015,0.00,No,Yes
5,Facebook,04-29-2011,N/A,7.58,Yes,No
6,Apple,10-15-2009,N/A,0.00,Yes,Yes
7,Airbnb,04-02-08,N/A,9.42,No,No
8,Nest,09-19-2006,N/A,3.00,Yes,Yes
9,Canon,03-07-05,09-20-2009,11.26,Yes,No


In [14]:
for i, row in df.iterrows():
    sql = "INSERT INTO csvtodbauto.customer_contracts VALUES (%s, %s, %s, %s, %s, %s)"
    cursor.execute(sql, tuple(row))
    print("Record inserted")

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


### Commit changes in database

In [15]:
db.commit()
cursor.close()
print('Table %s imported to database successfully.' % clean_tbl_name)

Table customer_contracts imported to database successfully.
